- Removal of similar sentences can be done with the built-in library difflib


In [48]:
import pandas as pd
import difflib
from tqdm import tqdm
from copy import deepcopy

tqdm.pandas()

In [9]:
def find_close_matches(string,list_of_strings):
    pass

In [31]:
path_to_jsonl_file = '../finetune/covid19_emergency_event/train.jsonl' 

df = pd.read_json(path_to_jsonl_file, lines=True)

In [33]:
print('The corpus contains {example_n} examples.'.format(example_n = df.shape[0]))

The corpus contains 3501 examples.


In [34]:
%%time
candidates = df.text.tolist()
df['matches']=df.text.progress_apply(lambda x: difflib.get_close_matches(x, candidates, n=len(candidates),cutoff = 0.8))

100%|███████████████████████████████████████| 3501/3501 [06:29<00:00,  8.99it/s]

CPU times: user 6min 18s, sys: 5.57 s, total: 6min 24s
Wall time: 6min 29s


In [35]:
df.head()

,text,event1,event2,event3,event4,event5,event6,event7,event8,matches
0,que ce principe implique que lorsqu'un risque ...,False,False,False,False,False,False,False,False,[que ce principe implique que lorsqu'un risque...
1,Ai concessionari di servizi di trasporto aereo...,False,True,False,False,False,False,False,False,[Ai concessionari di servizi di trasporto aere...
2,"VISTA la legge regionale 18 febbraio 2002, n. ...",False,False,False,False,False,False,False,False,"[VISTA la legge regionale 18 febbraio 2002, n...."
3,Voor de onder 4 tot en met 7 genoemde maatrege...,False,False,False,False,False,False,False,False,[Voor de onder 4 tot en met 7 genoemde maatreg...
4,2)ograniczenie obowiązku świadczenia pracy prz...,False,False,False,True,False,False,False,False,[2)ograniczenie obowiązku świadczenia pracy pr...


In [54]:
_id=1000
print('For the following sentence sentence:\n\n','\t',df.at[_id,'text'],'\n\n')
print('We have the following matches','\n\n')
for m in df.at[_id,'matches']:
    print('\t',m)
    print('---------------')

For the following sentence sentence:

 	 wprowadza się następujące zmiany: 1)w §2 :a)po ust.1dodaje się ust.1a wbrzmieniu:„1a.Kontrolę graniczną osób, przywróconą tymczasowo na okres, októrym mowa wust.1, przedłuża się na okres od dnia 25marca 2020r. do dnia 13kwietnia 2020r.” 


We have the following matches 


	 wprowadza się następujące zmiany: 1)w §2 :a)po ust.1dodaje się ust.1a wbrzmieniu:„1a.Kontrolę graniczną osób, przywróconą tymczasowo na okres, októrym mowa wust.1, przedłuża się na okres od dnia 25marca 2020r. do dnia 13kwietnia 2020r.”
---------------


For the purpose of pretraining I propose we keep the longest example and remove all other cases

In [47]:

sorted(df.at[_id,'matches'],key=lambda x: len(x),reverse=True)

["Vu l'avis des Ministres qui en ont délibéré en Conseil, donné le 10 juillet 2020;",
 "Vu l'avis des Ministres qui en ont délibéré en Conseil, donné le 17 avril 2020;",
 "Vu l'avis des Ministres qui en ont délibéré en Conseil, donné le 24 mars 2020 ;",
 "Vu l'avis des Ministres qui en ont délibéré en Conseil, donné le 15 mai 2020 ;",
 "Vu l'avis des Ministres qui en ont délibéré en Conseil, donné le 29 mai 2020 ;"]

In [51]:
df_new = deepcopy(df)
for i, _ in df.iterrows():
    matches = df.at[i,'matches']
    matches = sorted(matches,key=lambda x: len(x),reverse=True)
    if len(matches)>1:
        df_new = df_new[df_new.text.isin(matches[1:])==False]
        

In [63]:
print('The new corpus now contains {example_n} examples.'.format(example_n = df_new.shape[0]))

The new corpus now contains 2694 examples.


Performing a similarity search should not yield any results

In [58]:
%%time
candidates = df_new.text.tolist()
df_new['matches']=df_new.text.progress_apply(lambda x: difflib.get_close_matches(x, candidates, n=len(candidates),cutoff = 0.8))

100%|███████████████████████████████████████| 2694/2694 [03:33<00:00, 12.62it/s]

CPU times: user 3min 28s, sys: 2.56 s, total: 3min 30s
Wall time: 3min 33s


In [62]:
df_new[df_new.matches.apply(lambda x:len(x))>1]

,text,event1,event2,event3,event4,event5,event6,event7,event8,matches


## Notes
- For very large datasets we might want to use a slightly different approach, because of the large data size which might not fit into memory
